In [1]:
#pip install tensorflow

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## RANDOM FOREST MODEL

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [14]:
param_grid = {
    'n_estimators': [50, 100, 200],  # You can adjust this range
    'max_depth': [None, 10, 20, 30]  # You can adjust this range
}

In [15]:
# Create the Random Forest model
rf = RandomForestClassifier()

In [16]:
# Create the GridSearchCV object
grid_search = GridSearchCV(rf, param_grid, cv=5, verbose=2, error_score='raise')

In [17]:
# Fit the GridSearchCV object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ....................max_depth=None, n_estimators=50; total time= 3.0min
[CV] END ....................max_depth=None, n_estimators=50; total time= 3.7min
[CV] END ....................max_depth=None, n_estimators=50; total time= 4.6min
[CV] END ....................max_depth=None, n_estimators=50; total time= 3.9min
[CV] END ....................max_depth=None, n_estimators=50; total time= 3.1min
[CV] END ...................max_depth=None, n_estimators=100; total time= 6.0min
[CV] END ...................max_depth=None, n_estimators=100; total time= 8.7min
[CV] END ...................max_depth=None, n_estimators=100; total time= 7.5min
[CV] END ...................max_depth=None, n_estimators=100; total time= 6.5min
[CV] END ...................max_depth=None, n_estimators=100; total time= 5.9min
[CV] END ...................max_depth=None, n_estimators=200; total time= 9.4min
[CV] END ...................max_depth=None, n_es

GridSearchCV(cv=5, error_score='raise', estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'n_estimators': [50, 100, 200]},
             verbose=2)

In [32]:
# Get the best estimator and evaluate it
best_rf = grid_search.best_estimator_
y_test_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Random Forest Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Best Random Forest Parameters: {'max_depth': 20, 'n_estimators': 100}
Accuracy: 95.21%


In [33]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,220.008030,35.238795,0.588009,0.177572,None,50,"{'max_depth': None, 'n_estimators': 50}",0.958333,0.952991,0.961538,0.956197,0.962607,0.958333,0.003511,10
1,415.423393,62.267270,0.506163,0.192391,None,100,"{'max_depth': None, 'n_estimators': 100}",0.957265,0.956197,0.959402,0.957265,0.964744,0.958974,0.003067,3
2,761.159391,171.836560,0.969455,0.303376,None,200,"{'max_depth': None, 'n_estimators': 200}",0.957265,0.954060,0.959402,0.957265,0.964744,0.958547,0.003537,8
3,104.035125,15.803345,0.540009,0.161112,10,50,"{'max_depth': 10, 'n_estimators': 50}",0.957265,0.955128,0.958333,0.960470,0.961538,0.958547,0.002281,8
4,174.348962,31.754278,0.528640,0.163441,10,100,"{'max_depth': 10, 'n_estimators': 100}",0.957265,0.956197,0.959402,0.956197,0.964744,0.958761,0.003212,4
5,337.598424,39.390408,0.590378,0.173153,10,200,"{'max_depth': 10, 'n_estimators': 200}",0.957265,0.955128,0.960470,0.957265,0.963675,0.958761,0.002991,4
6,211.109701,2.278083,0.685487,0.018973,20,50,"{'max_depth': 20, 'n_estimators': 50}",0.959402,0.955128,0.955128,0.955128,0.962607,0.957479,0.003052,11
7,279.061983,22.587378,0.403480,0.027612,20,100,"{'max_depth': 20, 'n_estimators': 100}",0.958333,0.956197,0.960470,0.958333,0.963675,0.959402,0.002528,1
8,649.706038,118.650901,0.737296,0.174742,20,200,"{'max_depth': 20, 'n_estimators': 200}",0.956197,0.954060,0.960470,0.958333,0.964744,0.958761,0.003676,4
9,243.250555,42.393281,0.663310,0.140833,30,50,"{'max_depth': 30, 'n_estimators': 50}",0.957265,0.952991,0.958333,0.955128,0.962607,0.957265,0.003241,12


In [34]:
df[['param_n_estimators', 'params', 'mean_test_score']]

,param_n_estimators,params,mean_test_score
0,50,"{'max_depth': None, 'n_estimators': 50}",0.958333
1,100,"{'max_depth': None, 'n_estimators': 100}",0.958974
2,200,"{'max_depth': None, 'n_estimators': 200}",0.958547
3,50,"{'max_depth': 10, 'n_estimators': 50}",0.958547
4,100,"{'max_depth': 10, 'n_estimators': 100}",0.958761
5,200,"{'max_depth': 10, 'n_estimators': 200}",0.958761
6,50,"{'max_depth': 20, 'n_estimators': 50}",0.957479
7,100,"{'max_depth': 20, 'n_estimators': 100}",0.959402
8,200,"{'max_depth': 20, 'n_estimators': 200}",0.958761
9,50,"{'max_depth': 30, 'n_estimators': 50}",0.957265


In [35]:
grid_search.best_score_

0.9594017094017093

In [36]:
grid_search.best_params_

{'max_depth': 20, 'n_estimators': 100}

In [23]:
# import matplotlib.pyplot as plt

# k_values = [ 3, 5, 7, 9, 11]  # List of K values
# accuracies = [ 0.943803, 0.942521, 0.941880, 0.941453, 0.939530]  # Corresponding accuracies

# plt.figure(figsize=(8, 6))
# plt.plot(k_values, accuracies, marker='o')
# plt.xlabel('K (Number of Neighbors)')
# plt.ylabel('Accuracy')
# plt.title('Accuracy vs. K')
# plt.grid(True)
# plt.show()

In [24]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# y_test_pred = grid_search.predict(X_test)
# # Calculate the confusion matrix
# cm = confusion_matrix(y_test, y_test_pred)

# # Create a heatmap of the confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# plt.title("Confusion Matrix")
# plt.show()

In [25]:
# from sklearn.model_selection import cross_val_score

# cv_scores = cross_val_score(grid_search, X, y, cv=5)  # Perform 5-fold cross-validation

# plt.figure(figsize=(8, 6))
# plt.bar(range(1, 6), cv_scores)
# plt.xlabel('Fold')
# plt.ylabel('Accuracy')
# plt.title('Cross-Validation Results')
# plt.grid(axis='y')
# plt.show()